# Automated ML

we start by importing all the dependencies that we will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import json
import joblib


## Dataset

### Overview
We first of all define the workspace and create the experiment 'Parkinson-classification-AutoML' 

We then access to the dataset from the external link (placed in the github)

We move then to prepare the data(read the data using TabularDatasetFactory  then transforme it to a dataframe then store it in a csv file ) and send it to the default datastore so that we can use it in the Automl Config 

To finally create the compute cluster 

The dataset used in this notebook  is composed of a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD). Each column in the data is a particular voice measure, and each row corresponds one of 195 voice recording from these individuals ("name" column)

In [2]:
#Connection to the workspace and definition of the experiment 
ws = Workspace.from_config()
experiment_name = 'Parkinson-classification-AutoML'

experiment=Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-132695
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-132695


In [3]:
#Get the data from github
path_url = 'https://raw.githubusercontent.com/hananeouhammouch/Parkinsons-detection/master/parkinsons.data'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)
ds.to_pandas_dataframe().head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
#Prepare the data and send it to the datastore
df=ds.to_pandas_dataframe()
df.drop("name", inplace=True, axis=1)  
df.to_csv("./training_data.csv")
datastore = ws.get_default_datastore()
datastore.upload(src_dir = "./", target_path = "data/")
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_data.csv"))])


Uploading an estimated of 8 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 8
Uploading ./hyperparam_tuning.ipynb
Uploaded ./hyperparam_tuning.ipynb, 2 files out of an estimated total of 8
Uploading ./train.py
Uploaded ./train.py, 3 files out of an estimated total of 8
Uploading ./training_data.csv
Uploaded ./training_data.csv, 4 files out of an estimated total of 8
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-31-21-13-16.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-31-21-13-16.ipynb, 5 files out of an estimated total of 8
Uploading ./.ipynb_aml_checkpoints/hyperparam_tuning-checkpoint2020-11-31-21-11-55.ipynb
Uploaded ./.ipynb_aml_checkpoints/hyperparam_tuning-checkpoint2020-11-31-21-11-55.ipynb, 6 files out of an estimated total of 8
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 7 files out of an estimated total of 8
Uploading ./.ipynb_checkp

In [28]:
df.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.99,157.30,75.00,0.01,0.00,0.00,0.01,0.01,0.04,0.43,...,0.07,0.02,21.03,1,0.41,0.82,-4.81,0.27,2.30,0.28
1,122.40,148.65,113.82,0.01,0.00,0.00,0.01,0.01,0.06,0.63,...,0.09,0.02,19.09,1,0.46,0.82,-4.08,0.34,2.49,0.37
2,116.68,131.11,111.56,0.01,0.00,0.01,0.01,0.02,0.05,0.48,...,0.08,0.01,20.65,1,0.43,0.83,-4.44,0.31,2.34,0.33
3,116.68,137.87,111.37,0.01,0.00,0.01,0.01,0.02,0.05,0.52,...,0.09,0.01,20.64,1,0.43,0.82,-4.12,0.33,2.41,0.37
4,116.01,141.78,110.66,0.01,0.00,0.01,0.01,0.02,0.06,0.58,...,0.10,0.02,19.65,1,0.42,0.82,-3.75,0.23,2.33,0.41


In [5]:
#Creation of the compute-cluster
amlcompute_cluster_name = "cpu-cluster"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=5)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Creating
Succeeded.....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

In this section we configure the automnl setting and submit the experiment 

And Bellow is the description/Reason for the configuration used in Automl

|Setting |Reasons ?|
|-|-|
|**experiment_timeout_minutes**|Maximum amount of time in minutes that all iterations combined can take before the experiment terminates (15 minute because the dataset include only 195 lines)|
|**max_concurrent_iterations**|To help manage child runs  in parallele mode and when they can be performed, we create a dedicated cluster per experiment, and match the number of this setting (4) to the number of nodes in the cluster(5-1))|
|**n_cross_validations**|Number of cross validation (5) splits to ensure that they will be no overfiting |
|**primary_metric**|This is the metric that we want to optimize (accuracy) |
|**task**|classification |
|**compute_target**|To define the compute cluster we will be using |
|**training_data**|To specify the training dataset stored in the datastore  |
|**label_column_name**|To specify the dependent variable that we are trying to classify |

In [6]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes" :15,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

# Automl config 
automl_config = AutoMLConfig(
    task="classification",
    compute_target=aml_compute,
    training_data=training_data,
    label_column_name="status",
    **automl_settings
)

In [7]:
# Submit experiment
auto_ml_run = experiment.submit(config = automl_config, show_output = True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_15b2b046-252e-4a41-80fa-7fda6fffe1a0

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values we

## Run Details

`RunDetails` widgets used here to show the different experiments.

In [8]:
RunDetails(auto_ml_run).show()
auto_ml_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_15b2b046-252e-4a41-80fa-7fda6fffe1a0',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-31T21:32:58.586327Z',
 'endTimeUtc': '2020-12-31T21:58:23.852484Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Parkinson-classification-AutoML","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-132695","workspace_name":"quick-starts-ws-132695","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"nu

## Best Model

The best model from the automl experiments is retrieved and  all the properties of the model are displayed



In [9]:
# Retrieve the best automl model.
best_run, fitted_model = auto_ml_run.get_output()
print(best_run)
print(fitted_model)

#Register the model
description ='Parkinson detection classification '
model_name='Parkinson-detection-automl'
model_path='./'
tags = None
model = auto_ml_run.register_model(model_name = model_name, description = description , tags = tags)
print(auto_ml_run.model_id)

Run(Experiment: Parkinson-classification-AutoML,
Id: AutoML_15b2b046-252e-4a41-80fa-7fda6fffe1a0_0,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                 LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                                    colsample_bytree=1.0,
                                    importance_type='split', learning_rate=0.1,
                                

## Model Deployment


In the cell below we first  register the model, then define the environement to finally  create the inference config and deploy the model as a web service.

In [10]:
#Save the best model for the deployement
import os
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','automl_best_model_park.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [11]:
#Register the best model for the deployement

model=best_run.register_model(
            model_name = 'automl-bestmodel-parkinson', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description='Parkinson detection'
)

In [12]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [13]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_auto_park.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_auto_park.py',
                                    environment=myenv)

In [14]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [15]:
# deploye the model as a web service 
service = Model.deploy(workspace=ws, 
                       name='automl-bestmodel-parkinso', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [16]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-132695', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-132695'), name=automl-bestmodel-parkinso, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [17]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://5b2e14bf-750d-4de5-98dd-d00959f0d528.southcentralus.azurecontainer.io/score
Swagger URI:
http://5b2e14bf-750d-4de5-98dd-d00959f0d528.southcentralus.azurecontainer.io/swagger.json


In [51]:
# select 3  samples from the dataframe
x_df=df.sample(3)
y_df = x_df.pop('status')

x_df['Column1'] = 0.0

# convert the records to a json data file
recored=x_df.to_dict(orient='records')

scoring_json = json.dumps({'data': recored})
print(scoring_json)                         

{"data": [{"MDVP:Fo(Hz)": 148.09, "MDVP:Fhi(Hz)": 162.824, "MDVP:Flo(Hz)": 67.343, "MDVP:Jitter(%)": 0.00762, "MDVP:Jitter(Abs)": 5e-05, "MDVP:RAP": 0.00467, "MDVP:PPQ": 0.00354, "Jitter:DDP": 0.014, "MDVP:Shimmer": 0.05428, "MDVP:Shimmer(dB)": 0.497, "Shimmer:APQ3": 0.03357, "Shimmer:APQ5": 0.02494, "MDVP:APQ": 0.03635, "Shimmer:DDA": 0.1007, "NHR": 0.02431, "HNR": 21.718, "RPDE": 0.487407, "DFA": 0.727313, "spread1": -6.261141, "spread2": 0.120956, "D2": 2.137075, "PPE": 0.141958, "Column1": 0.0}, {"MDVP:Fo(Hz)": 128.451, "MDVP:Fhi(Hz)": 150.449, "MDVP:Flo(Hz)": 75.632, "MDVP:Jitter(%)": 0.01551, "MDVP:Jitter(Abs)": 0.00012, "MDVP:RAP": 0.00905, "MDVP:PPQ": 0.00909, "Jitter:DDP": 0.02716, "MDVP:Shimmer": 0.0617, "MDVP:Shimmer(dB)": 0.584, "Shimmer:APQ3": 0.03223, "Shimmer:APQ5": 0.03672, "MDVP:APQ": 0.05174, "Shimmer:DDA": 0.09669, "NHR": 0.11843, "HNR": 15.06, "RPDE": 0.639808, "DFA": 0.643327, "spread1": -4.20273, "spread2": 0.310163, "D2": 2.638279, "PPE": 0.356881, "Column1": 0.0

Send a request to the web deployed service  to test it.

In [52]:
output = service.run(scoring_json)

The result of the model vs the expectation

In [53]:
print (output)

{"result": [1, 1, 0]}


In [54]:
print(y_df)

69     1
101    1
166    0
Name: status, dtype: int64


In the cell below, the logs of the web service are printed once over the service is deleted

In [49]:
print(service.get_logs())


2020-12-31T22:18:28,809853837+00:00 - iot-server/run 
2020-12-31T22:18:28,809983939+00:00 - rsyslog/run 
2020-12-31T22:18:28,814131779+00:00 - gunicorn/run 
2020-12-31T22:18:28,818435521+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

to delete the service and the compute target

In [56]:

service.delete()
aml_compute.delete()

No service with name automl-bestmodel-parkinso found to delete.
